In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [2]:
df = pd.read_csv('train.csv')


In [3]:
import pandas as pd

# Supondo que 'data' seja o seu DataFrame e 'engine_capacity' seja o nome da coluna
# Substitua 'data' e 'engine_capacity' com os seus dados reais
valores_unicos = df['mileage'].unique()

print("Valores únicos na coluna 'engine_capacity':")
print(valores_unicos)


Valores únicos na coluna 'engine_capacity':
['23.84 kmpl' '17.5 kmpl' '17.4 kmpl' '14.08 kmpl' '21.4 kmpl'
 '17.66 kmpl' '17.52 kmpl' '16.09 kmpl' '22.48 kmpl' '18.24 kmpl'
 '21.74 kmpl' '19.27 kmpl' '23.01 kmpl' '21.5 kmpl' '22.38 kmpl' '18 kmpl'
 '22.41 kmpl' '11 kmpl' '27.39 kmpl' '16.55 kmpl' '19 kmpl' '23.59 kmpl'
 '15.1 kmpl' '18.6 kmpl' nan '20.36 kmpl' '16.38 kmpl' '18.2 kmpl'
 '22.77 kmpl' '17 kmpl' '17.6 kmpl' '18.15 kmpl' '19.34 kmpl' '11.3 kmpl'
 '20.7 kmpl' '22.9 kmpl' '14.84 kmpl' '20.71 kmpl' '17.7 kmpl' '15.8 kmpl'
 '17.05 kmpl' '18.16 kmpl' '16 kmpl' '18.7 kmpl' '14.8 kmpl' '15.74 kmpl'
 '18.3 kmpl' '15.6 kmpl' '15.68 kmpl' '18.9 kmpl' '24.2 kmpl' '14.11 kmpl'
 '16.35 kmpl' '22.05 kmpl' '17.01 kmpl' '23.95 kmpl' '19.17 kmpl'
 '17.8 kmpl' '20 kmpl' '26.1 kmpl' '19.67 kmpl' '12.8 kmpl' '22.07 kmpl'
 '26.2 kmpl' '18.06 kmpl' '14.5 kmpl' '34.05 kmpl' '20.51 kmpl' '33 kmpl'
 '13.68 kmpl' '16.36 kmpl' '25.8 kmpl' '17.3 kmpl' '21.21 kmpl'
 '16.47 kmpl' '13.9 kmpl' '23 kmpl' '

In [4]:
# SEATS __________________________________________________
distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)

df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI']), 'seats'] = 5
df.loc[df['full_name'] == '2008 Ford Endeavour 3.0 4x4 Thunder Plus', 'seats'] = 7

distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)


# ENGINE __________________________________________________
df['engine_capacity'] = df['engine_capacity'].str.replace('cc', '')


# KMS __________________________________________________
df['kms_driven'] = df['kms_driven'].str.replace('Kms', '')
df['kms_driven'] = df['kms_driven'].str.replace(',', '').str.strip()

# MAX POWER 
Correct_Max_Power = []
for power in df['max_power']:
    if pd.notna(power):
        if isinstance(power, str):
            if 'bhp' in power:
                num_bhp = float(re.search(r'\d+.?\d', power).group())
                Correct_Max_Power.append(num_bhp)
            elif 'PS' in power:
                num_ps = float(re.search(r'\d+.?\d', power).group())
                num_bhp = round(num_ps * 0.98632, 2)
                Correct_Max_Power.append(num_bhp)
            else:
                num = re.search(r'\d+\.?\d*', power)
                Correct_Max_Power.append(float(num.group()) if num else None)
        else:
            Correct_Max_Power.append(power)
    else:
        Correct_Max_Power.append(None)

Correct_Max_Power = [float(value) if value is not None else value for value in Correct_Max_Power]
df['max_power'] = Correct_Max_Power



# MILEAGE __________________________________________________
consumo_correto = []

for i in df['mileage']:
    match = re.search(r'(\d+.\d+|\d+)', str(i))

    if match:
        valor = float(match.group())

        if 'km/kg' in str(i):
            valor *= 1.40

        consumo_correto.append(valor if pd.notnull(i) else None)

while len(consumo_correto) < len(df):
    consumo_correto.append(None)

df['mileage'] = consumo_correto

media_mileage = df['mileage'].astype(float).mean()
df['mileage'].fillna(media_mileage, inplace=True)

# OWNER TYPE __________________________________________________

# Substituir os valores em branco ('') pela moda ('mode') da coluna 'owner_type'
moda_owner_type = df['owner_type'].mode()[0]  # Calcula a moda
df['owner_type'].replace('', moda_owner_type, inplace=True)

# INSURANCE __________________________________________________
#Deletar "insurance"
df['insurance'] = df['insurance'].str.replace('insurance', '')

#Substituir valores em branco e valores numéricos por 'No Insurance'
df['insurance'] = df['insurance'].apply(lambda x: 'No Insurance' if pd.isnull(x) or pd.api.types.is_numeric_dtype(x) else x)

#Corrigir "Third Party" com espaço extra
df['insurance'] = df['insurance'].str.replace('Third Party ', 'Third Party')







Distinct full names with blank seats:
2009 Maruti Estilo LXI
2019 Nissan Kicks Diesel
2008 Ford Endeavour 3.0 4x4 Thunder Plus
2010 Maruti Estilo LXI

Distinct full names with blank seats:


In [5]:
# Extract the year and create a new column 'ano'
df['ano'] = df['full_name'].str.extract(r'^(\d{4})')

# Display the resulting DataFrame for 'ano'
print(df['ano'])

# Extract 'marca' with patterns 'word' or 'word-word' and create a new column
df['marca'] = df['full_name'].str.extract(r'\d{4}\s(\w+(?:-\w+)?)')

# Display the resulting DataFrame for 'marca'
print(df['marca'])

# Get everything after 'marca' for the 'modelo'
df['modelo'] = df['full_name'].str.extract(r'\d{4}\s\w+\s(.+)')

# If 'modelo' is null, fill it with the contents of 'full_name'
df['modelo'].fillna(df['full_name'], inplace=True)

# Remove 'marca' from 'modelo'
df['modelo'] = df.apply(lambda x: x['modelo'].replace(x['marca'], ''), axis=1)

# Print the resulting DataFrame
print(df[['ano', 'marca', 'modelo']])








0        2019
1        2018
2        2015
3        2021
4        2019
         ... 
13951    2021
13952    2017
13953    2017
13954    2017
13955    2015
Name: ano, Length: 13956, dtype: object
0           Tata
1          Honda
2          Honda
3           Tata
4         Maruti
          ...   
13951        Kia
13952    Hyundai
13953      Skoda
13954     Maruti
13955      Honda
Name: marca, Length: 13956, dtype: object
        ano    marca                                         modelo
0      2019     Tata                                  Tiago XZ Plus
1      2018    Honda                                 WR-V i-VTEC VX
2      2015    Honda                                 City i VTEC SV
3      2021     Tata  New Safari XZA Plus Adventure Edition AT BSVI
4      2019   Maruti                            Baleno 1.2 CVT Zeta
...     ...      ...                                            ...
13951  2021      Kia                               Carnival Premium
13952  2017  Hyundai             

In [6]:
contagem_nulos = df.isnull().sum()

print(f"O número de valores nulos na coluna é: {contagem_nulos}")

O número de valores nulos na coluna é: full_name             0
registered_year      54
engine_capacity       9
insurance             0
transmission_type     0
kms_driven            3
owner_type           36
fuel_type             0
max_power            89
seats                 0
mileage               0
body_type             0
city                  0
resale_price_Lakh     0
ano                   0
marca                 0
modelo                0
dtype: int64


In [7]:
#BODY TYPE ----------------------
#Replace the missing types and the ones that were wrong
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI', '2019 Maruti Wagon R ZXI AMT 1.2BSIV',
                             '2016 Mercedes-Benz B Class B180 Sport', '2016 Mercedes-Benz A Class A200 D Sport Edition',
                             '2004 Tata Indigo Marina GLX', '2017 Volvo V40 D3 R Design']), 'body_type'] = 'Hatchback'
df.loc[df['full_name'].isin(['2005 Tata Sumo Victa EX 10/7 Str BSII', '2008 Tata Sumo CX', '2010 Tata New Safari DICOR 2.2 EX 4x2', 
                             '2017 Mahindra KUV 100 mFALCON G80 K8', '2019 Mahindra Bolero Camper GOLD ZX 2WD']), 'body_type'] = 'SUV'
df.loc[df['full_name'].isin(['2015 Honda Mobilio E i-DTEC', '2018 Maruti Eeco 5 Seater AC BSIV', '2016 Datsun GO Plus A']), 'body_type'] = 'MUV'
df.loc[df['full_name'].isin(['2021 BMW 7 Series 730Ld DPE Signature','2006 Hyundai Accent CRDi', '2007 Maruti SX4 Vxi BSIII',
                             '2020 Mercedes-Benz S-Class Maybach S560', '2004 Skoda Octavia 1.8 Turbo Petrol RS MT']), 'body_type'] = 'Sedan'
df.loc[df['full_name'].isin(['2012 Mercedes-Benz SLK-Class SLK 350', '2013 Mercedes-Benz E-Class E350 Cabriolet', 
                             '2009 Mercedes-Benz SLK-Class 55 AMG']), 'body_type'] = 'Convertibles'
df.loc[df['full_name'] == '2011 Jaguar XK R V8 Coupe', 'body_type'] = 'Coupe'
df.loc[df['full_name'].isin(['2015 Chevrolet Enjoy 1.4 LS 7', '2004 Maruti Versa DX2 8-SEATER BSIII TWIN A/C', '2021 Maruti Eeco 5 STR With AC Plus HTR CNG',
                             '2022 Maruti Eeco 5 Seater AC 2020-2022', '2022 Maruti Eeco 5 Seater AC BSVI', '2017 Maruti Eeco Flexi Green', 
                             '2013 Maruti Omni Limited Edition', '2021 Maruti Eeco CNG 5 Seater AC', '2019 Maruti Eeco CNG 5 Seater AC BSIV', 
                             '2019 Maruti Eeco 5 Seater AC', '2013 Maruti Eeco 5 Seater Standard BSIV', '2015 Maruti Omni CNG', 
                             '2015 Maruti Omni MPI CARGO BSIV', '2016 Maruti Eeco CNG 5 Seater AC BSIV', '2013 Toyota Innova 2.5 E Diesel MS 7-seater',
                             '2015 Chevrolet Enjoy 1.4 LS 7']), 'body_type'] = 'Minivans'
df.loc[df['full_name'].isin(['2014 Tata Xenon XT EX 4X2', '2019 Isuzu D-Max High']), 'body_type'] = 'Pickup'

# Check
distinct_full_names_with_blank_body_type = df[df['body_type'].isnull()]['full_name'].unique()
print('Distinct full names with blank body_type:')
for full_name in distinct_full_names_with_blank_body_type:
    print(full_name)

Distinct full names with blank body_type:


In [8]:
# Mudar formatos

df = df.astype({
    'seats': float,
    'engine_capacity': float,
    'kms_driven': float,
    'max_power': float,
    'ano': float,
    'marca': 'string',
    'modelo': 'string',
    'body_type': 'string',
    'mileage': float,
})

In [9]:

novo_valor = df['mileage'].mean()
df['mileage'] = df['mileage'].fillna(novo_valor)

In [10]:
novo_valor = df['engine_capacity'].mean()
df['engine_capacity'] = df['engine_capacity'].fillna(novo_valor)

In [11]:
novo_valor = df['max_power'].mean()
df['max_power'] = df['max_power'].fillna(novo_valor)

In [12]:
novo_valor = df['kms_driven'].mean()
df['kms_driven'] = df['kms_driven'].fillna(novo_valor)

In [13]:
# Check data types
data_types = df.dtypes
print("Data type by column:")
print(data_types)

Data type by column:
full_name                    object
registered_year              object
engine_capacity             float64
insurance                    object
transmission_type            object
kms_driven                  float64
owner_type                   object
fuel_type                    object
max_power                   float64
seats                       float64
mileage                     float64
body_type            string[python]
city                         object
resale_price_Lakh           float64
ano                         float64
marca                string[python]
modelo               string[python]
dtype: object


In [14]:
df = df.dropna(subset=['registered_year'])
# MARCA __________________________________________________
#df = df.dropna(subset=['marca'])"
# MODELO __________________________________________________
#df = df.dropna(subset=['modelo'])

In [15]:
# Change formats

df = df.astype({
    'seats': float,
    'engine_capacity': float,
    'kms_driven': float,
    'max_power': float,
    'ano': float,
    'marca': 'string',
    'modelo': 'string',
   
})

In [16]:
# Obter os valores únicos da coluna 'marca'
valores_unicos_marca = df['marca'].unique()

# Exibir os valores únicos da coluna 'marca'
print(valores_unicos_marca)



<StringArray>
[         'Tata',         'Honda',        'Maruti',           'BMW',
       'Hyundai',    'Volkswagen',       'Renault', 'Mercedes-Benz',
      'Mahindra',          'Ford',        'Toyota',            'MG',
        'Jaguar',        'Datsun',     'Chevrolet',           'Kia',
          'Land',        'Nissan',         'Skoda',    'Mitsubishi',
         'Volvo',          'Audi',       'Citroen',          'Jeep',
         'Force',          'Fiat',         'Lexus',          'Mini',
         'Isuzu',     'OpelCorsa',       'Porsche',       'Bentley',
        'Daewoo']
Length: 33, dtype: string


In [17]:
# Encontre o valor mínimo, máximo, média e moda da coluna 'resale_price_Lakh'
valor_minimo = df['resale_price_Lakh'].min()
valor_maximo = df['resale_price_Lakh'].max()
media = df['resale_price_Lakh'].mean()
moda = df['resale_price_Lakh'].mode()[0]  # O método mode retorna uma série, pegamos o primeiro valor

print(f"Valor mínimo: {valor_minimo}")
print(f"Valor máximo: {valor_maximo}")
print(f"Média: {media}")
print(f"Moda: {moda}")


Valor mínimo: 0.28
Valor máximo: 225.0
Média: 8.816425788375774
Moda: 4.5


In [18]:
# groups marca

limites = [0,20, df['resale_price_Lakh'].max()]
rotulos = [ 'baixa','alta']

df['categoria_marca'] = pd.cut(df.groupby('marca')['resale_price_Lakh'].transform('mean'), bins=limites, labels=rotulos, right=False)



In [19]:
# Filtrar o DataFrame para exibir os valores na coluna 'marca' quando 'modelo' é nulo
marca_modelo_nulo = df.loc[df['categoria_marca'].isnull(), 'marca']

# Exibir os valores na coluna 'marca' quando 'modelo' é nulo
print(marca_modelo_nulo)


Series([], Name: marca, dtype: string)


In [20]:
# Verificar valores nulos em todo o DataFrame
nulos_df = df.isnull().sum()

# Exibir a contagem de valores nulos por coluna
print(nulos_df)

full_name            0
registered_year      0
engine_capacity      0
insurance            0
transmission_type    0
kms_driven           0
owner_type           0
fuel_type            0
max_power            0
seats                0
mileage              0
body_type            0
city                 0
resale_price_Lakh    0
ano                  0
marca                0
modelo               0
categoria_marca      0
dtype: int64


In [21]:
df.columns

Index(['full_name', 'registered_year', 'engine_capacity', 'insurance',
       'transmission_type', 'kms_driven', 'owner_type', 'fuel_type',
       'max_power', 'seats', 'mileage', 'body_type', 'city',
       'resale_price_Lakh', 'ano', 'marca', 'modelo', 'categoria_marca'],
      dtype='object')

In [22]:
#Export
#Not the target
colunas_selecionadas = ['full_name', 'city','engine_capacity','insurance','transmission_type', 'categoria_marca','kms_driven','ano','marca','modelo','mileage','seats','engine_capacity','body_type','fuel_type','kms_driven','max_power','owner_type','resale_price_Lakh']


if set(colunas_selecionadas).issubset(df.columns):
    df_selecionado = df[colunas_selecionadas]
    df_selecionado.to_csv('Cenario9Sem_IDS.csv', index=False)

In [23]:
df.columns

Index(['full_name', 'registered_year', 'engine_capacity', 'insurance',
       'transmission_type', 'kms_driven', 'owner_type', 'fuel_type',
       'max_power', 'seats', 'mileage', 'body_type', 'city',
       'resale_price_Lakh', 'ano', 'marca', 'modelo', 'categoria_marca'],
      dtype='object')